In [78]:
import glob
import os
import pandas as pd
import numpy as np
import math
import scipy.stats as st
import yaml
import biom

import qiime2
import qiime2.FeatureTable

ModuleNotFoundError: No module named 'qiime2.FeatureTable'

In [50]:
ids = ['sample1', 'sample2', 'sample3']
df = pd.DataFrame(index=pd.Series(ids, name='ids'))
df
                  

""
ids
sample1
sample2
sample3


In [ ]:
!qiime decontam split-samples --help

In [72]:
!qiime tools import \
--input-path feature_table_test.biom \
--type 'FeatureTable[Frequency]' \
--input-format BIOMV210Format \
--output-path feature_table_test.qza

Imported feature_table_test.biom as BIOMV210Format to feature_table_test.qza


In [74]:
!qiime decontam split-samples \
--i-table feature_table_test.qza \
--m-sample-metadata-file split-batches-test.tsv \
--p-batch-types extraction \
--o-feature-tables-by-batch test.qza

Saved FeatureTableBatches[Frequency] to: test.qza


In [82]:
!unzip test.qza

Archive:  test.qza
  inflating: 8f2a67c6-992f-4dda-88d5-650c5b45b7bd/metadata.yaml  
  inflating: 8f2a67c6-992f-4dda-88d5-650c5b45b7bd/checksums.md5  
  inflating: 8f2a67c6-992f-4dda-88d5-650c5b45b7bd/VERSION  
  inflating: 8f2a67c6-992f-4dda-88d5-650c5b45b7bd/provenance/metadata.yaml  
  inflating: 8f2a67c6-992f-4dda-88d5-650c5b45b7bd/provenance/citations.bib  
  inflating: 8f2a67c6-992f-4dda-88d5-650c5b45b7bd/provenance/VERSION  
  inflating: 8f2a67c6-992f-4dda-88d5-650c5b45b7bd/provenance/artifacts/741025c8-19f7-4efd-a1ba-9c9d2888bc93/metadata.yaml  
  inflating: 8f2a67c6-992f-4dda-88d5-650c5b45b7bd/provenance/artifacts/741025c8-19f7-4efd-a1ba-9c9d2888bc93/citations.bib  
  inflating: 8f2a67c6-992f-4dda-88d5-650c5b45b7bd/provenance/artifacts/741025c8-19f7-4efd-a1ba-9c9d2888bc93/VERSION  
  inflating: 8f2a67c6-992f-4dda-88d5-650c5b45b7bd/provenance/artifacts/741025c8-19f7-4efd-a1ba-9c9d2888bc93/action/action.yaml  
  inflating: 8f2a67c6-992f-4dda-88d5-650c5b45b7bd/provenance/artifact

In [81]:
test = qiime2.Artifact.load('test.qza')
biom_iterator = test.view(test.format).batches.iter_views(biom.Table) #generator - store the list
list(biom_iterator) #assign to variable or list above can call [0] to get first obj
# artifact has a method view
# test.view(test.format) - conversion into dir fomat, has property batches - the model file dir
# iter_views - generation results - iterator yeilding results
# test calls batches on what the helper func returns

[(PosixPath('extraction_1.0.biom'),
  0 x 0 <class 'biom.table.Table'> with 0 nonzero entries (0% dense)),
 (PosixPath('extraction_2.0.biom'),
  0 x 0 <class 'biom.table.Table'> with 0 nonzero entries (0% dense))]

In [66]:
extraction_biom = biom.Table(np.array([[100, 0, 0], [0, 0, 0], [100, 0, 0], [10, 10, 10], [10, 10, 0]]),
                                ['ASV_1', 'ASV_2', 'ASV_3', 'ASV_4', 'ASV_5'], 
                                ['sample1', 'sample2', 'sample3'])
print(extraction_biom)

# Constructed from biom file
#OTU ID	sample1	sample2	sample3
ASV_1	100.0	0.0	0.0
ASV_2	0.0	0.0	0.0
ASV_3	100.0	0.0	0.0
ASV_4	10.0	10.0	10.0
ASV_5	10.0	10.0	0.0


In [33]:
!biom convert -i two_feature_table.biom -o two_feature_table_from_biom.txt --to-tsv

In [43]:
!biom convert -i batch4_table_test.tsv -o batch4_table_test.biom --table-type="OTU table" --to-hdf5

In [132]:
md = (pd.DataFrame(
           {'sample_type': ['experimental', 'control', 'control', 
                            'experimental', 'control', 'control'],
            'amplification_type': ['experimental', 'experimental', 'control', 
                                   'experimental', 'experimental', 'control'],
            'extraction': ['1', '1', np.nan, '2', '2', np.nan],
            'extraction_type': ['experimental', 'control', 'experimental', 
                                   'experimental', 'control', 'experimental'],
            'amplification': ['1', '1', '1', '2', '2', '2']},
           index=pd.Index(['sample1', 'sample2', 'sample3', 'sample4',
                           'sample5', 'sample6'], name='sampleid')))

md

,sample_type,amplification_type,extraction,extraction_type,amplification
sampleid,,,,,
sample1,experimental,experimental,1,experimental,1
sample2,control,experimental,1,control,1
sample3,control,control,NaN,experimental,1
sample4,experimental,experimental,2,experimental,2
sample5,control,experimental,2,control,2
sample6,control,control,NaN,experimental,2


In [129]:
extraction_1 = biom.Table(
    np.array([[100, 0], [0, 0], [100, 0], [10, 10], [10, 10]]),
              ['ASV_1', 'ASV_2', 'ASV_3', 'ASV_4', 'ASV_5'],
              ['sample1', 'sample2'])
with biom.util.biom_open('q2_decontam/tests/data/batches/extraction_1.biom', 'w') as f:
    extraction_1.to_hdf5(f, "manual test data")

extraction_2 = biom.Table(
    np.array([[100, 0], [100, 0], [0, 0], [10, 10], [10, 10]]),
              ['ASV_1', 'ASV_2', 'ASV_3', 'ASV_4', 'ASV_5'],
              ['sample4', 'sample5'])
with biom.util.biom_open('q2_decontam/tests/data/batches/extraction_2.biom', 'w') as f:
    extraction_1.to_hdf5(f, "manual test data")

amplification_1 = biom.Table(
    np.array([[100, 0, 0], [0, 0, 0], [100, 0, 0],
              [10, 10, 10], [10, 10, 0]]),
              ['ASV_1', 'ASV_2', 'ASV_3', 'ASV_4', 'ASV_5'],
              ['sample1', 'sample2', 'sample3'])
with biom.util.biom_open('q2_decontam/tests/data/batches/amplification_1.biom', 'w') as f:
    extraction_1.to_hdf5(f, "manual test data")

amplification_2 = biom.Table(
    np.array([[100, 0, 0], [100, 0, 0], [0, 0, 0],
              [10, 10, 10], [10, 10, 0]]),
              ['ASV_1', 'ASV_2', 'ASV_3', 'ASV_4', 'ASV_5'],
              ['sample4','sample5', 'sample6'])
with biom.util.biom_open('q2_decontam/tests/data/batches/amplification_2.biom', 'w') as f:
    extraction_1.to_hdf5(f, "manual test data")

In [127]:
test_table = biom.load_table('q2_decontam/tests/data/batches/extraction_1.biom')
print(test_table)

# Constructed from biom file
#OTU ID	sample1	sample2
ASV_1	100.0	0.0
ASV_2	0.0	0.0
ASV_3	100.0	0.0
ASV_4	10.0	10.0
ASV_5	10.0	10.0


In [103]:
amplification_2 = biom.Table(
            np.array([[100, 0, 0], [100, 0, 0],
                      [0, 0, 0], [10, 10, 10],
                      [10, 10, 0]]),
                      ['ASV_1', 'ASV_2', 'ASV_3', 'ASV_4', 'ASV_5'],
                      ['sample4','sample5', 'sample6'])
print(amplification_2)

# Constructed from biom file
#OTU ID	sample4	sample5	sample6
ASV_1	100.0	0.0	0.0
ASV_2	100.0	0.0	0.0
ASV_3	0.0	0.0	0.0
ASV_4	10.0	10.0	10.0
ASV_5	10.0	10.0	0.0


In [113]:
!pwd

/Users/haley/Dropbox/qiime2_dev/git/q2-decontam


In [114]:
!ls

8f2a67c6-992f-4dda-88d5-650c5b45b7bd makefile
LICENSE.txt                          q2_decontam
batch1_table_test.tsv                q2_decontam.egg-info
batch2_table_test.tsv                setup.py
batch3_table_test.tsv                split-batches-test.tsv
batch4_table_test.tsv                test.qza
decontam_testing.ipynb               test_batches.qza
feature_table_test-Copy1.biom        thing.py
feature_table_test.biom              two_feature_table.biom
feature_table_test.qza               two_feature_table_from_biom.txt
licenses


In [118]:
!cd q2_decontam/tests/data

In [116]:
!pwd

/Users/haley/Dropbox/qiime2_dev/git/q2-decontam


In [48]:
dict = {'amplification_1': ['sample1', 'sample2', 'sample3']}
# print(yaml.safe_dump(dict, default_flow_style=False))
with open ('amp_1_test.yml', 'w') as outfile:
    yaml.safe_dump(dict, outfile, default_flow_style = False)

In [ ]:
data = q2.Metadata(pd.DataFrame(
          {'extraction': [1,1,np.nan ,2 ,2,np.nan],
          'amplification': ['1', '1', '1', '2', '2', '2']},
          index=pd.Index(['sample1', 'sample2', 'sample3', 'sample4', 'sample5', 'sample6'], name='sampleid')))
print(data.view(Dataframe))

In [19]:
test = data.isnull()
test

,extraction,amplification
sampleid,,
sample1,False,False
sample2,False,False
sample3,True,False
sample4,False,False
sample5,False,False
sample6,True,False


In [23]:
print(test.iat[5,0])

True


In [24]:
data2 = pd.read_csv('/Users/haley/Dropbox/qiime2_dev/git/q2-decontam/q2_decontam/tests/data/split-batches-test.tsv', sep = '\t')
data2

,sampleid,extraction,amplification
0,sample1,1.0,1
1,sample2,1.0,1
2,sample3,NaN,1
3,sample4,2.0,2
4,sample5,2.0,2
5,sample6,NaN,2


In [131]:
import random

l = ['Alex', 'Charissa', 'Conor', 'Grace', 'Haley', 'Justin', 'Conor', 'Vennesa', 'Elisa']
l2 = ['Alex', 'Grace', 'Justin', 'Madeline']
print(random.sample(l2, 4))




['Justin', 'Alex', 'Madeline', 'Grace']


In [ ]:
import random

l = ['Ankita', 'Conor', 'Haley']

print(random.sample(l, 1))


In [8]:
!qiime decontam hello --help

Usage: qiime decontam hello [OPTIONS]

  print 'hello world'

Outputs:
  --o-text ARTIFACT    
    Greeting                                                        [required]
Miscellaneous:
  --output-dir PATH    Output unspecified results to a directory
  --verbose / --quiet  Display verbose output to stdout and/or stderr during
                       execution of this action. Or silence output if
                       execution is successful (silence is golden).
  --examples           Show usage examples and exit.
  --citations          Show citations and exit.
  --help               Show this message and exit.


In [5]:
!make run

qiime decontam hello --o-text example_greeting --verbose
Saved Greeting to: example_greeting.qza


def prepare control batches (feature table, metadata, name of sample_type col, name of experimental samples in sample_type col (default to experimental, optional parameter name_of_experimental_sample = 'experimental'))
    create a list of unique values for each control type metadata col sample_type
    use filter samples to create a list of samples where sample_type = each unique value 
    return lists of sample ids associated with each sample_type except
    use filter features on returned lists keep only features present in at least 2 samples were sample_type = experimental
    return subsets of feature tables with features in 2 or more experimental samples named by sample_type
    
    look at my py docs
    
    from typing import List, Tuple
        List[biom.Table] are mutable

        Tuple[biom.Table, ...] immutable 
        
        ar_1, var2 = some_func_that_returns a two-tuple
        var_1 first value from the tuple 
base class - underlying functionality - don't use
concrete - childern of base classes - concrete implimentation 

https://stackoverflow.com/questions/13957829/how-to-use-raise-keyword-in-python/13957915#13957915

https://docs.pytest.org/en/6.2.x/goodpractices.html#test-discovery

q2 lint
BSD3 clause license
regex to check header against some standard for q2
require that q2 is in header 